<a href="https://colab.research.google.com/github/MonishaBasak/Creating-a-Superhero-and-Supervillain-Name-Generator-Using-Tensorflow/blob/main/Superhero_Name_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Superhero (and Supervillain) Name Generator

---

[Superhero Names Dataset](https://github.com/am1tyadav/superhero)

## Task 2

1. Import the data
2. Create a tokenizer
3. Char to index and Index to char dictionaries

In [ ]:
!git clone https://github.com/am1tyadav/superhero

fatal: destination path 'superhero' already exists and is not an empty directory.


In [ ]:
with open('superhero/superheroes.txt','r') as f:
  data= f.read()

  data[:100]

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.8.2


In [ ]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(
    filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~',
    split='\n',
)

In [ ]:
tokenizer.fit_on_texts(data)

In [ ]:
char_to_index = tokenizer.word_index
index_to_char = dict((v,k)for k, v in char_to_index.items())

print(index_to_char)

{1: '\t', 2: 'a', 3: 'e', 4: 'r', 5: 'o', 6: 'n', 7: 'i', 8: ' ', 9: 't', 10: 's', 11: 'l', 12: 'm', 13: 'h', 14: 'd', 15: 'c', 16: 'u', 17: 'g', 18: 'k', 19: 'b', 20: 'p', 21: 'y', 22: 'w', 23: 'f', 24: 'v', 25: 'j', 26: 'z', 27: 'x', 28: 'q'}


## Task 3

1. Converting between names and sequences

In [ ]:
names = data.splitlines()
names[:10]

['jumpa\t',
 'doctor fate\t',
 'starlight\t',
 'isildur\t',
 'lasher\t',
 'varvara\t',
 'the target\t',
 'axel\t',
 'battra\t',
 'changeling\t']

In [ ]:
tokenizer.texts_to_sequences(names[0])

[[25], [16], [12], [20], [2], [1]]

In [ ]:
def name_to_seq(name):
  return [tokenizer.texts_to_sequences(c)[0][0] for c in name]

In [ ]:
name_to_seq(names[1])

[14, 5, 15, 9, 5, 4, 8, 23, 2, 9, 3, 1]

In [ ]:
def seq_to_name(seq):
  return ''.join([index_to_char[i] for i in seq if i !=0 ])

In [ ]:
seq_to_name(name_to_seq(names[1]))

'doctor fate\t'

## Task 4

1. Creating sequences
2. Padding all sequences

In [ ]:
sequences = []

for name in names:
  seq = name_to_seq(name)
  if len(seq) >= 2:
    sequences += [seq[:i] for i in range (2, len(seq) + 1)]

In [ ]:
sequences[:15]

[[25, 16],
 [25, 16, 12],
 [25, 16, 12, 20],
 [25, 16, 12, 20, 2],
 [25, 16, 12, 20, 2, 1],
 [14, 5],
 [14, 5, 15],
 [14, 5, 15, 9],
 [14, 5, 15, 9, 5],
 [14, 5, 15, 9, 5, 4],
 [14, 5, 15, 9, 5, 4, 8],
 [14, 5, 15, 9, 5, 4, 8, 23],
 [14, 5, 15, 9, 5, 4, 8, 23, 2],
 [14, 5, 15, 9, 5, 4, 8, 23, 2, 9],
 [14, 5, 15, 9, 5, 4, 8, 23, 2, 9, 3]]

In [ ]:
max_len = max([len(x) for x in sequences])
print(max_len)

33


In [ ]:
padded_sequences= tf.keras.preprocessing.sequence.pad_sequences(sequences, padding='pre', maxlen= max_len)

print(padded_sequences[0])

[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0 25 16]


In [ ]:
padded= tf.keras.preprocessing.sequence.pad_sequences(sequences, padding='post', maxlen= max_len)

print(padded[0])

[25 16  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0]


In [ ]:
padded_sequences.shape

(88279, 33)

## Task 5: Creating Training and Validation Sets

1. Creating training and validation sets

In [ ]:
x, y= padded_sequences[:, :-1], padded_sequences[:, -1]
print(x.shape, y.shape)

(88279, 32) (88279,)


In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test =train_test_split(x,y)

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(66209, 32) (66209,)
(22070, 32) (22070,)


In [ ]:
num_chars = len(char_to_index.keys())+1
print(num_chars)

29


## Task 6: Creating the Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D,MaxPool1D, LSTM, Bidirectional,Dense



In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D,MaxPool1D, LSTM, Bidirectional,Dense

model= Sequential([
    Embedding(num_chars, 8, input_length= max_len-1),
                   Conv1D(64, 5, strides=1, activation='tanh', padding='causal'),
                   MaxPool1D(2),
                   LSTM(32),
                   Dense(num_chars, activation='softmax')
                   ])
model.compile(
             loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy']
)
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 32, 8)             232       
                                                                 
 conv1d_6 (Conv1D)           (None, 32, 64)            2624      
                                                                 
 max_pooling1d_6 (MaxPooling  (None, 16, 64)           0         
 1D)                                                             
                                                                 
 lstm_6 (LSTM)               (None, 32)                12416     
                                                                 
 dense_6 (Dense)             (None, 29)                957       
                                                                 
Total params: 16,229
Trainable params: 16,229
Non-trainable params: 0
__________________________________________________

## Task 7: Training the Model

In [ ]:
r = model.fit( x_train, y_train, 
              validation_data=(x_test, y_test), epochs=50,
              verbose=2, callbacks= [tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=3)])

Epoch 1/50
2070/2070 - 28s - loss: 2.7358 - accuracy: 0.1918 - val_loss: 2.5499 - val_accuracy: 0.2358 - 28s/epoch - 13ms/step
Epoch 2/50
2070/2070 - 12s - loss: 2.5151 - accuracy: 0.2437 - val_loss: 2.4748 - val_accuracy: 0.2526 - 12s/epoch - 6ms/step
Epoch 3/50
2070/2070 - 12s - loss: 2.4527 - accuracy: 0.2572 - val_loss: 2.4322 - val_accuracy: 0.2624 - 12s/epoch - 6ms/step
Epoch 4/50
2070/2070 - 14s - loss: 2.4102 - accuracy: 0.2706 - val_loss: 2.4015 - val_accuracy: 0.2751 - 14s/epoch - 7ms/step
Epoch 5/50
2070/2070 - 13s - loss: 2.3750 - accuracy: 0.2804 - val_loss: 2.3776 - val_accuracy: 0.2775 - 13s/epoch - 6ms/step
Epoch 6/50
2070/2070 - 13s - loss: 2.3436 - accuracy: 0.2931 - val_loss: 2.3547 - val_accuracy: 0.2884 - 13s/epoch - 6ms/step
Epoch 7/50
2070/2070 - 13s - loss: 2.3150 - accuracy: 0.3014 - val_loss: 2.3351 - val_accuracy: 0.2986 - 13s/epoch - 6ms/step
Epoch 8/50
2070/2070 - 14s - loss: 2.2889 - accuracy: 0.3087 - val_loss: 2.3177 - val_accuracy: 0.3062 - 14s/epoch - 

## Task 8: Generate Names!

In [ ]:
def generate_names(seed):
  for i in range(0,40):
    seq = name_to_seq(seed)
    pad= tf.keras.preprocessing.sequence.pad_sequences([seq], padding='pre', maxlen=max_len-1,
                                                       truncating='pre')
    pred= model.predict(pad)[0]
    pred_char= index_to_char[tf.argmax(pred).numpy()]
    seed += pred_char

    if pred_char == '\t':
      break
  print(seed)

In [ ]:
generate_names('mon')

montor doom	
